<h1><center>Real-Time Location System Case Study</center></h1>
<center> David Tran, Frank Sclafani, Matthew Przybyla, and Brandon Hill</center>
<center> MSDS 7333 - Quantifying the World - Case Study 6 (Unit 6)</center>
<center> June 19, 2018</center>

In [1]:
# This cell just tells the notebook to set the width of the cells in relation to the screen width
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 80%; }
    div#menubar-container     { width: 55%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""))

import warnings
warnings.filterwarnings('ignore')

In [33]:
import pandas as pd
print('Pulling Raw Data')

rtls_link = 'http://rdatasciencecases.org/Data/offline.final.trace.txt'
series = pd.read_table(rtls_link, squeeze=True, engine='python', skiprows=2, skipfooter=1)
series.tail()

print('Raw Data Loaded')

Pulling Raw Data
Raw Data Loaded


In [37]:
labels = ['T','ID','POS','DEGREE','MAC_0','MAC_1','MAC_2','MAC_3','MAC_4',
          'MAC_5','MAC_6','MAC_7','MAC_8','MAC_9','MAC_10','MAC_11','MAC_12',
          'MAC_13','MAC_14','MAC_15','MAC_16','MAC_17','MAC_18','MAC_19','MAC_20']
df = pd.DataFrame([x.split(';') for x in series.values], columns = labels)

In [43]:
df['T'] = df['T'].str.replace("t=", "")
df['ID'] = df.ID.str.replace("id=", "")
df['POS'] = df.POS.str.replace("pos=", "")
df['DEGREE'] = df.DEGREE.str.replace("degree=", "")

In [45]:
d = {}
for i, x in enumerate(df['POS'].values):
    if x != None:
        temp = x.split(",")
        d[i] = [float(y) for y in temp]
    else:
        d[i] = None

pos = pd.Series(d)

df['POS'] = pos

In [49]:
df['DEGREE'] = pd.to_numeric(df.DEGREE, errors='coerce', downcast='float')

In [50]:
df.tail()

,T,ID,POS,DEGREE,MAC_0,MAC_1,MAC_2,MAC_3,MAC_4,MAC_5,...,MAC_11,MAC_12,MAC_13,MAC_14,MAC_15,MAC_16,MAC_17,MAC_18,MAC_19,MAC_20
151383,1141936869448,00:02:2D:21:0F:33,"[20.0, 8.0, 0.0]",315.700012,"00:0f:a3:39:e1:c0=-51,2462000000,3","00:14:bf:b1:97:8d=-56,2442000000,3","00:14:bf:b1:97:8a=-60,2437000000,3","00:14:bf:b1:97:90=-69,2427000000,3","00:0f:a3:39:dd:cd=-76,2412000000,3","02:b7:00:bb:a9:35=-67,2447000000,1",...,None,None,None,None,None,None,None,None,None,None
151384,1141936869700,00:02:2D:21:0F:33,"[20.0, 8.0, 0.0]",315.700012,"00:0f:a3:39:e1:c0=-53,2462000000,3","00:14:bf:3b:c7:c6=-52,2432000000,3","00:14:bf:b1:97:8d=-62,2442000000,3","00:14:bf:b1:97:81=-58,2422000000,3","00:14:bf:b1:97:8a=-61,2437000000,3","00:0f:a3:39:dd:cd=-77,2412000000,3",...,None,None,None,None,None,None,None,None,None,None
151385,1141936869952,00:02:2D:21:0F:33,"[20.0, 8.0, 0.0]",315.700012,"00:0f:a3:39:e1:c0=-54,2462000000,3","00:14:bf:3b:c7:c6=-58,2432000000,3","00:14:bf:b1:97:8d=-56,2442000000,3","00:14:bf:b1:97:8a=-60,2437000000,3","00:14:bf:b1:97:90=-69,2427000000,3","00:0f:a3:39:dd:cd=-77,2412000000,3",...,None,None,None,None,None,None,None,None,None,None
151386,1141936870204,00:02:2D:21:0F:33,"[20.0, 8.0, 0.0]",315.700012,"00:14:bf:3b:c7:c6=-51,2432000000,3","00:0f:a3:39:e1:c0=-53,2462000000,3","00:14:bf:b1:97:8a=-61,2437000000,3","00:14:bf:b1:97:81=-60,2422000000,3","00:14:bf:b1:97:8d=-62,2442000000,3","00:0f:a3:39:dd:cd=-77,2412000000,3",...,None,None,None,None,None,None,None,None,None,None
151387,1141936870456,00:02:2D:21:0F:33,"[20.0, 8.0, 0.0]",315.700012,"00:14:bf:3b:c7:c6=-56,2432000000,3","00:0f:a3:39:e1:c0=-53,2462000000,3","00:14:bf:b1:97:8d=-61,2442000000,3","00:14:bf:b1:97:8a=-60,2437000000,3","00:14:bf:b1:97:81=-60,2422000000,3","00:14:bf:b1:97:90=-68,2427000000,3",...,None,None,None,None,None,None,None,None,None,None


# Notebook Navigation

[1) Abstract](#Abstract) 

[2) Introduction](#Introduction)

[3) Methods](#Methods - Signal Frontier Analysis) 

[4) Results](#Results) 

[5) Future Work / Discussion / Conclusions](#Future Work / Discussion / Conclusions) 

[6) References](#References) 

# Abstract

Predicting Location via Indoor Positioning Systems


    


# Introduction

## Background

The stock market volumes dataset collected via the Yahoo Finance API contains 1,304 observations and 6 variables. There is no missing data within any of the variables across all observations. Our objective is to analyze the historical daily adjusted closing stock prices for 5 major retail companies for the 5-year span between June 4, 2013 and June 1, 2018. A full list of variable descriptions, including the Datetime variable in which the observation was captured and the data is indexed by, can be found in Table 1:

|    Variable   |                                         Description                                         |    Type    |
|:-------------:|:-------------------------------------------------------------------------------------------:|:----------:|
| DatetimeIndex | Date observation was collected. (Index of data)                                             |  Ordinal   |
| AMZN          | Amazon.com, Inc. adjusted close price adjusted for both dividends and   splits.             | Continuous |
| WMT           | Walmart Inc. adjusted close price adjusted for both dividends and splits.                   | Continuous |
| TGT           | Target Corporation adjusted close price adjusted for both dividends and   splits.           | Continuous |
| BBY           | Best Buy Co., Inc. adjusted close price adjusted for both dividends and   splits.           | Continuous |
| COST          | Costco Wholesale Corporation adjusted close price adjusted for both   dividends and splits. | Continuous |

In [3]:
px.describe()

NameError: name 'px' is not defined

# Methods - Signal Frontier Analysis

# Results

# Future Work / Discussion / Conclusions

# References